## Generate grid

In [36]:
import json
import geojson as gj
import shapely.geometry as sp
import folium
import logging
import math

from uuid import uuid4
from landez import MBTilesBuilder

### Open file

In [32]:
with open('/code/data/geojsons/abuja.geojson', 'r') as file:
    abuja = json.load(file)

## Generate grid given zoom level of 11

In [33]:
# Maximum resolution of OSM
MAXRESOLUTION = 156543.0339

# X/Y axis offset
AXIS_OFFSET = MAXRESOLUTION * 256 / 2

def degrees_to_meters(lon, lat):
    x = lon * AXIS_OFFSET / 180
    y = math.log(math.tan((90 + lat) * math.pi / 360)) / (math.pi / 180)
    y = y * AXIS_OFFSET / 180

    return x, y

def meters_to_degrees(x,y):
    lon = x * 180 / AXIS_OFFSET
    # thanks magichim @ github for the correction
    lat = math.atan(math.exp(y * math.pi / AXIS_OFFSET)) * 360 / math.pi - 90

    return lon, lat


def create_task_feature(step, x, y):
    xmin = x * step - AXIS_OFFSET
    ymin = y * step - AXIS_OFFSET
    xmax = (x + 1) * step - AXIS_OFFSET
    ymax = (y + 1) * step - AXIS_OFFSET
    
    minlnglat = meters_to_degrees(xmin, ymin)
    maxlnglat = meters_to_degrees(xmax, ymax)

    bbox = [minlnglat[0], minlnglat[1], maxlnglat[0], maxlnglat[1]]

    return bbox


def create_grid(bbox, zoom):
    xmin, ymin, xmax, ymax = math.ceil(bbox[0]), math.ceil(bbox[1]), math.floor(bbox[2]), math.floor(bbox[3])

    step = AXIS_OFFSET / (math.pow(2, (zoom - 1)))

    xminstep = int(math.floor((xmin + AXIS_OFFSET) / step))
    xmaxstep = int(math.ceil((xmax + AXIS_OFFSET) / step))
    yminstep = int(math.floor((ymin + AXIS_OFFSET) / step))
    ymaxstep = int(math.ceil((ymax + AXIS_OFFSET) / step))    
    task_features = []

    for x in range(xminstep, xmaxstep):
        for y in range(yminstep, ymaxstep):
            # print(x, ((2**zoom-1) - y) , zoom)
            task_feature = create_task_feature(step, x, y)
            task_features.append(task_feature)
    return task_features
    

def make_grid(bbox, zoom):
    minxy = degrees_to_meters(bbox[0], bbox[1])
    maxxy = degrees_to_meters(bbox[2], bbox[3])

    bbox = [minxy[0], minxy[1], maxxy[0], maxxy[1]]
    grid = create_grid(bbox, zoom)

    return grid


def get_bounds(poly):
    polygon = sp.Polygon(poly['geometry']['coordinates'][0])
    bounds = polygon.bounds
    
    return bounds
    
    
polygons = sp.MultiPolygon([sp.Polygon(p['geometry']['coordinates'][0]) for p in abuja['features']])
m = folium.Map(location=list(polygons.centroid.coords)[0], zoom_start=13)

features = []
for poly in abuja['features']:
    bounds = get_bounds(poly)
    grid = make_grid(bounds, 17)

    for item in grid:
        # Get coordinates.
        poly = sp.box(*item)
        list_polygon = list(poly.exterior.coords)
        
        # Create geojson.
        polygon = gj.Polygon([list_polygon])
        features.append(gj.Feature(geometry=polygon, properties={"id": uuid4().hex}))

        folium.Polygon(locations=list_polygon).add_to(m)

feature_collection = gj.FeatureCollection(features)

with open('myfile.geojson', 'w') as f:
   gj.dump(feature_collection, f)

In [38]:
mb = MBTilesBuilder(cache=False, tiles_headers={'User-Agent':'github.com/hotosm/osm-export-tool'})
for item in grid:
    mb.add_coverage(bbox=item, zoomlevels=[16, 17, 18])
mb.run()

DEBUG:landez.tiles:Clean-up /var/folders/yt/17rq21ns75d77_1n5jcysmc40000gn/T/landez/tiles
DEBUG:landez.tiles:Compute list of tiles for bbox [7.218017578125018, 8.860648239550983, 7.220764160156254, 8.863362033551695] on zooms [16, 17, 18].
DEBUG:landez.tiles:Add 6 tiles.
DEBUG:landez.tiles:6 tiles in total.
DEBUG:landez.tiles:Compute list of tiles for bbox [7.218017578125018, 8.863362033551695, 7.220764160156254, 8.866075807508224] on zooms [16, 17, 18].
DEBUG:landez.tiles:Add 6 tiles.
DEBUG:landez.tiles:12 tiles in total.
DEBUG:landez.tiles:Compute list of tiles for bbox [7.218017578125018, 8.866075807508224, 7.220764160156254, 8.868789561414701] on zooms [16, 17, 18].
DEBUG:landez.tiles:Add 6 tiles.
DEBUG:landez.tiles:17 tiles in total.
DEBUG:landez.tiles:Compute list of tiles for bbox [7.220764160156254, 8.860648239550983, 7.223510742187492, 8.863362033551695] on zooms [16, 17, 18].
DEBUG:landez.tiles:Add 6 tiles.
DEBUG:landez.tiles:22 tiles in total.
DEBUG:landez.tiles:Compute list

DEBUG:landez.tiles:tile method called with [16, 34082, 31147]
DEBUG:landez.sources:Download tile (16, 34082, 31147)
DEBUG:landez.sources:Retrieve tile at http://a.tile.openstreetmap.org/16/34082/31147.png
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): a.tile.openstreetmap.org:80
DEBUG:urllib3.connectionpool:http://a.tile.openstreetmap.org:80 "GET /16/34082/31147.png HTTP/1.1" 200 2286
DEBUG:landez.tiles:tile method called with [17, 68165, 62295]
DEBUG:landez.sources:Download tile (17, 68165, 62295)
DEBUG:landez.sources:Retrieve tile at http://c.tile.openstreetmap.org/17/68165/62295.png
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): c.tile.openstreetmap.org:80
DEBUG:urllib3.connectionpool:http://c.tile.openstreetmap.org:80 "GET /17/68165/62295.png HTTP/1.1" 200 2858
DEBUG:landez.tiles:tile method called with [17, 68164, 62296]
DEBUG:landez.sources:Download tile (17, 68164, 62296)
DEBUG:landez.sources:Retrieve tile at http://c.tile.openstreetmap.org/17/681

DEBUG:landez.sources:Download tile (18, 136328, 124593)
DEBUG:landez.sources:Retrieve tile at http://c.tile.openstreetmap.org/18/136328/124593.png
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): c.tile.openstreetmap.org:80
DEBUG:urllib3.connectionpool:http://c.tile.openstreetmap.org:80 "GET /18/136328/124593.png HTTP/1.1" 200 1990
DEBUG:landez.tiles:tile method called with [18, 136329, 124589]
DEBUG:landez.sources:Download tile (18, 136329, 124589)
DEBUG:landez.sources:Retrieve tile at http://c.tile.openstreetmap.org/18/136329/124589.png
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): c.tile.openstreetmap.org:80
DEBUG:urllib3.connectionpool:http://c.tile.openstreetmap.org:80 "GET /18/136329/124589.png HTTP/1.1" 200 103
DEBUG:landez.tiles:tile method called with [17, 68166, 62296]
DEBUG:landez.sources:Download tile (17, 68166, 62296)
DEBUG:landez.sources:Retrieve tile at http://b.tile.openstreetmap.org/17/68166/62296.png
DEBUG:urllib3.connectionpool:Startin

DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136330	Row (y): 137554
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136330	Row (y): 137555
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136331	Row (y): 137550
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136331	Row (y): 137551
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136331	Row (y): 137552
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136331	Row (y): 137553
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136331	Row (y): 137554
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136331	Row (y): 137555
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136332	Row (y): 137550
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136332	Row (y): 137551
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136332	Row (y): 137552
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (x): 136332	Row (y): 137553
DEBUG:mbutil.util: Read tile from Zoom (z): 18	Col (

In [23]:
list_polygon

[(7.226257324218764, 8.866075807508224),
 (7.226257324218764, 8.868789561414701),
 (7.223510742187492, 8.868789561414701),
 (7.223510742187492, 8.866075807508224),
 (7.226257324218764, 8.866075807508224)]